**formatPropertyMissing.ipynb**

Deprecated concept with disallowed annotations or axioms.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/formatPropertyMissing.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [16]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and all properties that are mandatory or recommended for Format concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concept is considered to be a format if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``formats``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#formats"/>`` or ``<oboInOwl:inSubset>formats />``

**NB.3:** Currently EDAM uses ``<documentation>`` property for 3 different types of Format attributes (see https://edamontologydocs.readthedocs.io/en/latest/developers_guide.html#mandatory-attributes) - this should be cleaned up, and the logic here refined.

**NB.4:** Many of the checks should not apply to "placeholder" Format concepts (see https://edamontologydocs.readthedocs.io/en/latest/technical_details.html#concept-types).  The code could be improved to take account of this.

**NB.5:** The code does not yet check that the type of data a Format concept specifies is encoded.  There are two ways this is supposed to be done, see https://edamontologydocs.readthedocs.io/en/latest/developers_guide.html#mandatory-attributes). For example ``<is_format_of>`` relationship (to some Data concept).  The complication is that these are modelled as OWL restrictions, or ``subClassOf`` relationships - which are tricky to handle.

In [17]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?subsetpath ?documentation ?file_extension ?media_type ?example ?information_standard ?ontology_used ?organisation WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :documentation ?documentation .}
OPTIONAL {?id :file_extension ?file_extension .}
OPTIONAL {?id :media_type ?media_type .}
OPTIONAL {?id :example ?example .}
OPTIONAL {?id :information_standard ?information_standard .}
OPTIONAL {?id :ontology_used ?ontology_used .}
OPTIONAL {?id :organisation ?organisation .}
}
"""

# Run SPARQL query and collate results
err = NOERR
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [18]:
# Error message building function
def report_append(err_new, msg):
    """
    Writes a report of an error and sets the error level appropriately.
    err: the error level to set
    msg: the message to write
    """
    global err
    global errfound
    global report
    if err_new > err:
        err = err_new
    errfound = True
    report.append(msg)

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])

    documentation = str(r['documentation'])
    file_extension = str(r['file_extension'])
    media_type = str(r['media_type'])
    example = str(r['example'])
    information_standard = str(r['information_standard'])
    ontology_used = str(r['ontology_used'])
    organisation = str(r['organisation'])


    # Check for Format concepts
    if "formats" in subsetpath:
        if documentation == "None":
            report_append(WARN, "WARN Missing property (documentation) on ::: " + id +  ' (' + term + ')')
        if file_extension == "None":
            report_append(WARN, "WARN Missing property (file_extension) on ::: " + id +  ' (' + term + ')')
        if media_type == "None":
            report_append(INFO, "INFO Missing property (media_type) on ::: " + id +  ' (' + term + ')')
        if example == "None":
            report_append(INFO, "INFO Missing property (example) on ::: " + id +  ' (' + term + ')')
        if information_standard == "None":
            report_append(INFO, "INFO Missing property (information_standard) on ::: " + id +  ' (' + term + ')')
        if ontology_used == "None":
            report_append(INFO, "INFO Missing property (ontology_used) on ::: " + id +  ' (' + term + ')')
        if organisation == "None":
            report_append(INFO, "INFO Missing property (organisation) on ::: " + id +  ' (' + term + ')')

Write report and return approriate value.

In [19]:
# Return exit code (raises exception)
if errfound:
    print('Invalid property found for these obsolete concepts:')
    print("\n".join(report))
    sys.exit(WARN)
else:
    print("No issues found.")
    sys.exit(NOERR)

Invalid property found for these obsolete concepts:
WARN Missing property (file_extension) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (media_type) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (example) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (information_standard) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (ontology_used) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (organisation) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
WARN Missing property (file_extension) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (media_type) on ::: http://edamontology.org/format_3820 (Relaxed PHYLIP Sequential)
INFO Missing property (example) on ::: http://edamontology.org/format_3820 (Rela

SystemExit: 2

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
